# Modelamiento del sistema

* **Temporalidad:** Discreta (observaciones diarias).

* **Demanda:**
  Sea $D$ la demanda diaria del medicamento.

  $$
  D \sim \text{Poisson}(\lambda = 3)
  $$

  $$
  P(D = k) = \frac{3^k e^{-3}}{k!}, \quad k = 0,1,2,\dots
  $$

  $$
  P(D \geq k) = \sum_{r=k}^{\infty} \frac{3^r e^{-3}}{r!}
  $$

* **Variable de estado:**

  $$
  X_n = \text{Número de unidades del medicamento disponibles al inicio del día } n
  $$

* **Espacio de estados:**

  $$
  S_X = \{0,1,2,3,4,5\}
  $$

* **Colaboración hospitalaria:**
  Si al final del día el inventario queda en **0**, la farmacia entra en cuarentena y el inventario del día siguiente depende de la colaboración de hospitales.

  $$
  Y \sim \text{Binomial}(n=5, p=0.07)
  $$

  $$
  P(Y = k) = \binom{5}{k}(0.07)^k (0.93)^{5-k}, \quad k=0,1,\dots,5
  $$

* **Política de transición:**

$$
p_{ij} =
\begin{cases}
\mathbb{P}(D = i - j), & i \geq 2,\ j \neq 0 \\
\mathbb{P}(D \geq i), & i \geq 2,\ j = 0 \\
\mathbb{P}(D = 5 - j), & 1 \leq i < 2,\ j \neq 0 \\
\mathbb{P}(D \geq 5), & 1 \leq i < 2,\ j = 0 \\
\mathbb{P}(Y = j), & i = 0,\ j \in \{0,1,\dots,5\} \\
0, & \text{d.l.c.}
\end{cases}
$$




In [ ]:
import numpy as np
from scipy.stats import poisson, binom
from jmarkov.dtmc import dtmc

: 

In [16]:
# Parametros del problema
lmbda = 3 
n = 5
p = 0.07
umbral = 2

In [ ]:
# Espacio de estados
estados = np.array([i for i in range(n+1)])

# Matriz de transicion
matriz = np.zeros((len(estados), len(estados)))

In [18]:
for i in range(len(estados)):
    for j in range(len(estados)):
        
        if i >= umbral and j != 0:
            matriz[i,j] = poisson.pmf(i-j, lmbda)
        elif i >= umbral and j == 0:
            matriz[i,j] = 1 - poisson.cdf(i-1, lmbda)
        elif 1 <= i < umbral and j != 0:
            matriz[i,j] = poisson.pmf(5-j, lmbda)
        elif 1 <= i < umbral and j == 0:
            matriz[i,j] = 1 - poisson.cdf(5-1, lmbda)
        elif i == 0 and j in estados:
            matriz[i,j] = binom.pmf(j, n, p)
        else:
            matriz[i,j] = 0
            
matriz

array([[6.95688369e-01, 2.61818204e-01, 3.94134930e-02, 2.96660700e-03,
        1.11646500e-04, 1.68070000e-06],
       [1.84736755e-01, 1.68031356e-01, 2.24041808e-01, 2.24041808e-01,
        1.49361205e-01, 4.97870684e-02],
       [8.00851727e-01, 1.49361205e-01, 4.97870684e-02, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [5.76809919e-01, 2.24041808e-01, 1.49361205e-01, 4.97870684e-02,
        0.00000000e+00, 0.00000000e+00],
       [3.52768111e-01, 2.24041808e-01, 2.24041808e-01, 1.49361205e-01,
        4.97870684e-02, 0.00000000e+00],
       [1.84736755e-01, 1.68031356e-01, 2.24041808e-01, 2.24041808e-01,
        1.49361205e-01, 4.97870684e-02]])

In [19]:
cadena = dtmc(matriz, estados)

### a. **¿Cuál es la probabilidad de que el inventario sea igual a 5 dentro de 2 días, si hoy hay 1 unidad?**

Queremos calcular:

$$
P(X_2 = 5 \mid X_0 = 1)
$$

Esto corresponde a una **transición de dos pasos**, es decir:

$$
P(X_2 = 5 \mid X_0 = 1) = \big[P^2\big]_{1,5}
$$



In [20]:
alpha = np.array([0,1,0,0,0,0]) # Hoy hay 1 unidad en inventario

probabilidades = cadena.transient_probabilities(n=2, alpha=alpha) # 2 dias

probabilidad = round(probabilidades[5],4)
print(f"La probabilidad de que al cabo de 2 dias haya 5 unidades en inventario es {probabilidad}")

La probabilidad de que al cabo de 2 dias haya 5 unidades en inventario es 0.0108


### b. **¿Qué pasaría si el umbral de reposición cambiara de 2 a 3?**
¿Se reduciría el riesgo de quiebre?

* **Nueva Política de transición:**

$$
p_{ij} =
\begin{cases}
\mathbb{P}(D = i - j), & i \geq 3,\ j \neq 0 \\
\mathbb{P}(D \geq i), & i \geq 3,\ j = 0 \\
\mathbb{P}(D = 5 - j), & 1 \leq i < 3,\ j \neq 0 \\
\mathbb{P}(D \geq 5), & 1 \leq i < 3,\ j = 0 \\
\mathbb{P}(Y = j), & i = 0,\ j \in \{0,1,\dots,5\} \\
0, & \text{d.l.c.}
\end{cases}
$$


In [21]:
steady_state_original = cadena.steady_state()
riesgo_quiebre_original = round(steady_state_original[0],4) 

umbral = 3

for i in range(len(estados)):
    for j in range(len(estados)):
        
        if i >= umbral and j != 0:
            matriz[i,j] = poisson.pmf(i-j, lmbda)
        elif i >= umbral and j == 0:
            matriz[i,j] = 1 - poisson.cdf(i-1, lmbda)
        elif 1 <= i < umbral and j != 0:
            matriz[i,j] = poisson.pmf(5-j, lmbda)
        elif 1 <= i < umbral and j == 0:
            matriz[i,j] = 1 - poisson.cdf(5-1, lmbda)
        elif i == 0 and j in estados:
            matriz[i,j] = binom.pmf(j, n, p)
        else:
            matriz[i,j] = 0    
matriz

array([[6.95688369e-01, 2.61818204e-01, 3.94134930e-02, 2.96660700e-03,
        1.11646500e-04, 1.68070000e-06],
       [1.84736755e-01, 1.68031356e-01, 2.24041808e-01, 2.24041808e-01,
        1.49361205e-01, 4.97870684e-02],
       [1.84736755e-01, 1.68031356e-01, 2.24041808e-01, 2.24041808e-01,
        1.49361205e-01, 4.97870684e-02],
       [5.76809919e-01, 2.24041808e-01, 1.49361205e-01, 4.97870684e-02,
        0.00000000e+00, 0.00000000e+00],
       [3.52768111e-01, 2.24041808e-01, 2.24041808e-01, 1.49361205e-01,
        4.97870684e-02, 0.00000000e+00],
       [1.84736755e-01, 1.68031356e-01, 2.24041808e-01, 2.24041808e-01,
        1.49361205e-01, 4.97870684e-02]])

In [ ]:
cadena = dtmc(matriz, estados)
steady_state_nuevo = cadena.steady_state()  
riesgo_quiebre_nuevo = round(steady_state_nuevo[0],4)

if riesgo_quiebre_nuevo < riesgo_quiebre_original:
    print(f"Disminuye el riesgo de quiebre de {riesgo_quiebre_original} a {riesgo_quiebre_nuevo}")
else:
    print("No disminuye el riesgo de quiebre")    

Disminuye el riesgo de quiebre de 0.5648 a 0.4759


### c. **¿Cuánto impacta la probabilidad de colaboración hospitalaria ($p = 0.07$) en la confiabilidad del sistema?**
¿Cómo cambiarían las probabilidades si esta subiera a $p = 0.2$?

In [23]:
p = 0.2

matriz_nueva = np.zeros((len(estados), len(estados)))


for i in range(len(estados)):
    for j in range(len(estados)):
        
        if i >= umbral and j != 0:
            matriz_nueva[i,j] = poisson.pmf(i-j, lmbda)
        elif i >= umbral and j == 0:
            matriz_nueva[i,j] = 1 - poisson.cdf(i-1, lmbda)
        elif 1 <= i < umbral and j != 0:
            matriz_nueva[i,j] = poisson.pmf(5-j, lmbda)
        elif 1 <= i < umbral and j == 0:
            matriz_nueva[i,j] = 1 - poisson.cdf(5-1, lmbda)
        elif i == 0 and j in estados:
            matriz_nueva[i,j] = binom.pmf(j, n, p)
        else:
            matriz_nueva[i,j] = 0    
matriz_nueva

cadena_nueva = dtmc(matriz_nueva, estados)

# Probabilidad de tener inventario despues de un dia para cada una de las matrices

alpha = np.array([1,0,0,0,0,0]) # Hoy hay 0 unidades en inventario

probabilidades_nueva = cadena_nueva.transient_probabilities(n=1, alpha=alpha) # 1 dia

probabilidades_vieja = cadena.transient_probabilities(n=1, alpha=alpha) # 1 dia

print(f"La probabilidad de tener inventario despues de un dia con p=0.07 es {round(1-probabilidades_vieja[0],4)}")
print(f"La probabilidad de tener inventario despues de un dia con p=0.2 es {round(1-probabilidades_nueva[0],4)}")


La probabilidad de tener inventario despues de un dia con p=0.07 es 0.3043
La probabilidad de tener inventario despues de un dia con p=0.2 es 0.6723
